In [45]:
from DataClean1 import PreProcessor
import pandas as pd
from timeit import default_timer as timer
import numpy as np
import pandas as pd
from math import isnan
from sklearn import preprocessing
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from loguru import logger

In [46]:
df=pd.read_csv('./LaqnData.csv')

In [47]:
df

,Site,Species,ReadingDateTime,Value,Units,Provisional or Ratified
0,EN5,NO,01/01/2022 00:00,0.7,ug m-3,R
1,EN5,NO,01/01/2022 00:15,0.5,ug m-3,R
2,EN5,NO,01/01/2022 00:30,0.3,ug m-3,R
3,EN5,NO,01/01/2022 00:45,NaN,ug m-3,R
4,EN5,NO,01/01/2022 01:00,NaN,ug m-3,R
...,...,...,...,...,...,...
105115,EN5,NOX,31/12/2022 22:45,12.2,ug m-3 as NO2,R
105116,EN5,NOX,31/12/2022 23:00,9.9,ug m-3 as NO2,R
105117,EN5,NOX,31/12/2022 23:15,9.6,ug m-3 as NO2,R
105118,EN5,NOX,31/12/2022 23:30,8.0,ug m-3 as NO2,R


In [55]:
def HandlingMissingData(df,num_strategy='mean',cat_strategy='knn',n_neighbors=3,null_threshold=0.1):
        num_cols = df.select_dtypes(include=np.number).columns
        cat_cols = [column for column in df.columns if column not in num_cols]
            
        for column in num_cols:
            null_val=df[column].isnull().mean()
            if(null_val!=0 and null_val<=null_threshold):
                print(f'{null_val}% NaN values found on column: {column}')
                df=df.dropna(subset=[column])
                df.reset_index(drop=True)

        imputer=None
        if(num_strategy=='knn'):
            imputer = KNNImputer(n_neighbors=n_neighbors)
        else:
            if num_strategy in ['mean','median','mode','most_frequent']:
                imputer = SimpleImputer(strategy=num_strategy)
            else:
                print('Invalid imputer strategy specified :{}\nDefault strategy Mean is applied',num_strategy)
                imputer = SimpleImputer(strategy='mean')
        print('imputation process started...')
        print(num_cols)
        print(cat_cols)
        for feature in num_cols:
            if df[feature].isna().sum().sum() != 0:
                try:
                    df_imputed = pd.DataFrame(imputer.fit_transform(np.array(df[feature]).reshape(-1, 1)))
                    if (df[feature].fillna(-9999) % 1  == 0).all():
                        df[feature] = df_imputed
                        # round back to INTs, if original data were INTs
                        df[feature] = df[feature].round()
                        df[feature] = df[feature].astype('Int64')                                        
                    else:
                        df[feature] = df_imputed
                except:
                    print('imputation failed for feature "{}"',feature)
        if(cat_strategy=='knn'):
            imputer = KNNImputer(n_neighbors=n_neighbors)
        elif(cat_strategy=='logreq'):
            df = PreProcessor.LogisticRegressionImputer(
                columns=cat_cols,
                df=df
            )
            return df
        else:
            imputer = SimpleImputer(strategy='most_frequent')
        
        for feature in cat_cols:
            if df[feature].isna().sum()!= 0:
                try:
                    mapping = dict()
                    mappings = {k: i for i, k in enumerate(df[feature].dropna().unique(), 0)}
                    mapping[feature] = mappings
                    df[feature] = df[feature].map(mapping[feature])

                    df_imputed = pd.DataFrame(imputer.fit_transform(np.array(df[feature]).reshape(-1, 1)), columns=[feature])    

                    # round to integers before mapping back to original values
                    df[feature] = df_imputed
                    df[feature] = df[feature].round()
                    df[feature] = df[feature].astype('Int64')  

                    # map values back to original
                    mappings_inv = {v: k for k, v in mapping[feature].items()}
                    df[feature] = df[feature].map(mappings_inv)
                except:
                    print('Imputation failed for feature "{}"',  feature)
        return df
    

In [56]:
newClass=PreProcessor()

In [57]:
df=newClass.RemoveIrrelevantColumn(df)
HandlingMissingData(df, num_strategy='mean', cat_strategy='knn', n_neighbors=3, null_threshold=0.05)
newClass.normalization(df)
newClass.encoding(df)


0 irrelavant columns found.
0.024315068493150686% NaN values found on column: Value
imputation process started...
Index(['Species', 'Value', 'Units'], dtype='object')
['ReadingDateTime']
Mean of normalized feature: Value   -4.433791e-17
dtype: float64
Standard deviation of normalized feature: Value    1.0
dtype: float64


,Species,ReadingDateTime,Value,Units
0,0,01/01/2022 00:00,-0.681847,0
1,0,01/01/2022 00:15,-0.685976,0
2,0,01/01/2022 00:30,-0.690105,0
3,0,01/01/2022 00:45,NaN,0
4,0,01/01/2022 01:00,NaN,0
...,...,...,...,...
105115,2,31/12/2022 22:45,-0.444456,1
105116,2,31/12/2022 23:00,-0.491934,1
105117,2,31/12/2022 23:15,-0.498127,1
105118,2,31/12/2022 23:30,-0.531155,1


In [58]:
from sklearn import preprocessing
def encoding(df):
        lable_encoder = preprocessing.LabelEncoder() 
        for column in df.columns:
            if df[column].dtype == 'object' and df[column].nunique()<3: #binary 
                df[column]=lable_encoder.fit_transform(df[column])
            elif df[column].dtype == 'object' and (df[column].nunique()>2 and df[column].nunique()<=12): #Multi-class 
                df[column]=lable_encoder.fit_transform(df[column])
            elif df[column].dtype == 'bool':
                df[column]=lable_encoder.fit_transform(df[column])
            else:
                pass
            return df

In [59]:
encoding(df)

,Species,ReadingDateTime,Value,Units
0,0,01/01/2022 00:00,-0.681847,0
1,0,01/01/2022 00:15,-0.685976,0
2,0,01/01/2022 00:30,-0.690105,0
3,0,01/01/2022 00:45,NaN,0
4,0,01/01/2022 01:00,NaN,0
...,...,...,...,...
105115,2,31/12/2022 22:45,-0.444456,1
105116,2,31/12/2022 23:00,-0.491934,1
105117,2,31/12/2022 23:15,-0.498127,1
105118,2,31/12/2022 23:30,-0.531155,1


In [60]:
def encoding(df):   
    lable_encoder = preprocessing.LabelEncoder() 
    for column in df.columns:
            if df[column].dtype == 'object' and df[column].nunique()<3: #binary 
                df[column]=lable_encoder.fit_transform(df[column])
            elif df[column].dtype == 'object' and (df[column].nunique()>2 and df[column].nunique()<=12): #Multi-class 
                df[column]=lable_encoder.fit_transform(df[column])
            elif df[column].dtype == 'bool':
                df[column]=lable_encoder.fit_transform(df[column])
            else:
                pass
    return df

In [61]:
encoding(df)

,Species,ReadingDateTime,Value,Units
0,0,01/01/2022 00:00,-0.681847,0
1,0,01/01/2022 00:15,-0.685976,0
2,0,01/01/2022 00:30,-0.690105,0
3,0,01/01/2022 00:45,NaN,0
4,0,01/01/2022 01:00,NaN,0
...,...,...,...,...
105115,2,31/12/2022 22:45,-0.444456,1
105116,2,31/12/2022 23:00,-0.491934,1
105117,2,31/12/2022 23:15,-0.498127,1
105118,2,31/12/2022 23:30,-0.531155,1
